# Session 4: CT Acquisition

## Aims of this Session
- Learn how to perform a simulated CT Acquisition
- Use reconstructed data to see how simulation paramaters affect quality
- Discover the effects of polychromatic beams on results
- Compare simulated samples to real data

In [ ]:
import k3d
from gvxrPython3 import gvxr
from gvxrPython3.utils import visualise

# Performing a CT Acquisiton
CT scanners are able to create 3D slices through an object by using multiple X-ray images (also known as projections, or radiographs) taken at various angles around a sample. The full details and terminology of how this reconstruction method works will be covered in tomorrow's training with CIL, for now we will treat it as a black-box function. The quality of projections directly impact the details in the final result, in this session we will take a look at what can effect our scan. *Some of the impacts on getting good scans are not obvious!*

## Two Case Studies

To better demonstrate what is going on, we will show two examples based on real-world use cases.

One use case is using an aluminium plate in a standard industrial/Lab CT machine, where we want results to measure hole diameters cut into the plate.

The second case study is looking at a parallel synchrotron beam to scan titanium fibers, and seeing how non-obvious beam properties can have a large impact on results.

---

# Aluminium Plate Case Study

In This case study you will learn:
- How to simulate a CT scan with gvxr using python
- How detector properties affect result quality
- How to automate this simulation process with json2gvxr

The aluminium plate phantom is a 50mmx50mm plate with cylindrical holes cut into the surface. The holes are in a total of _ different sizes and do not cut through the entire plate. We would like to measure the diameters of these holes using a lab CT machine, but first we want to know if results are possible in the best-case scenario, hence using simulations.

In [ ]:
# Load stl and display
plot = k3d.plot()
with open("input_data/phantoms/doga-plate.stl", "rb") as model:
    plot += k3d.stl(model.read(),color=0xfdea4f)
plot.display()

## Task 1: Setup

With your new-found knowledge of gvxr, setup the experiment environment as follows:

1. Fixed detector 1m away from the source with a size of 150mmx48mm with a pixel stride of 70um
2. The detector's lsf has been measured and stored in `input_data/lsf.npy`, display and apply it to the detector
3. A tungsten tube source with the current power at 10keV
4. The plate model `input_data/phantoms/doga-plate.stl` is in mm and should be made from aluminium, lying face-up and in focus.

In [ ]:
# GVXR Init
gvxr.createWindow(-1,True,"OPENGL")

In [ ]:

# Detector
gvxr.setDetectorPosition(0, 0.2, 0, "m")
gvxr.setDetectorNumberOfPixels(2140, 960)
gvxr.setDetectorPixelSize(120,120,"um")
gvxr.setDetectorUpVector(0, 0, 1)
# gvxr.setLSF()

# Source
gvxr.setSourcePosition(0,-0.8,0,"m")
gvxr.usePointSource()
# gvxr.setFocalSpot(0,0,20,0.02,"mm",3)
gvxr.addEnergyBinToSpectrum(10, "keV", 100)

# Plate
gvxr.removePolygonMeshesFromSceneGraph()
gvxr.loadMeshFile("plate","./input_data/phantoms/doga-plate.stl","mm")
gvxr.setElement("plate","Al")
gvxr.setDensity("plate",2.7,"g/cm3")
gvxr.displayScene()

In [ ]:
# Preview the environment;
visualise()

Now that the scene is setup, we'll take a look into reproducing a virtual scan of the plate, by taking x-ray images and rotating the sample.

---

# Titanium Fibre Case Study

In this case study you will learn:
- An easy way to compare real results with simulated results
- Show how 'almost' pure sources will change real results
- Learn how noise also effects results, and how to simulate it with gvxr

---

## Extension Task: Scanning your own model

Now that you've gotten used to using gvxr in different ways, select a model from input_data and perform a virtual scan.

Feel free to pick the simplest, or most complex multi-material models.
Multi-material models are guarenteed to provide more challenging results, and can be fun to play around with material composition.